For this exercise we see if we can improve MNIST to 99.5% accuracy or more by adding only a single convolutional layer and a single MaxPooling 2D layer to the model. We stop training once the accuracy goes above this amount. It should happen in less than 10 epochs, so it's ok to hard code the number of epochs for training, but our training must end once it hits the above metric. If it doesn't, then we'll need to redesign your callback. When 99.5% accuracy has been hit, we print out the string "Reached 99.5% accuracy so cancelling training!"


In [2]:
# Import libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
# Load the data

# Get only training set
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data() 

## Pre-processing the data

One important step when dealing with image data is to preprocess the data. During the preprocess step we can apply transformations to the dataset that will be fed into our convolutional neural network.

Here we will apply two transformations to the data:
- Reshape the data so that it has an extra dimension. The reason for this is that commonly we will use 3-dimensional arrays (without counting the batch dimension) to represent image data. The third dimension represents the color using RGB values. This data might be in black and white format so the third dimension doesn't really add any additional information for the classification process but it is a good practice regardless.
- Normalize the pixel values so that these are values between 0 and 1. You can achieve this by dividing every value in the array by the maximum.

Remember that these tensors are of type `numpy.ndarray` so you can use functions like [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) or [divide](https://numpy.org/doc/stable/reference/generated/numpy.divide.html) to complete the `reshape_and_normalize` function below:

In [4]:
print(training_images)

In [5]:
print(training_images.shape)

In [6]:
# reshape_and_normalize
def reshape_and_normalize(images):

    # Reshape the images to add an extra dimension
    images = np.reshape(images, (60000, 28, 28, 1))
    
    # Normalize pixel values
    images = np.divide(images, np.amax(images))

    return images

Let's test the function:

In [7]:
# Reload the images in case you run this cell multiple times
(training_images, _), _ = tf.keras.datasets.mnist.load_data() 

# Apply your function
training_images = reshape_and_normalize(training_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")


## Defining callback

Now we create the callback that will ensure that training will stop after an accuracy of 99.5% is reached:

In [8]:
# Remember to inherit from the correct class
class myCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('accuracy') >= 0.995):
                print("\nReached 99.5% accuracy so cancelling training!") 
                
                # Stop training once the above condition is met
                self.model.stop_training = True

## Convolutional Model

Finally, we write the `convolutional_model` function. This function should return our convolutional neural network. **The model should achieve an accuracy of 99.5% or more before 10 epochs to pass this assignment.**

In [9]:
def convolutional_model():    

    # Define the model
    model = tf.keras.models.Sequential([ 
        
    # Add convolutions and max pooling
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),  
    
    # Add the same layers as before
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
    ])     

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

In [10]:
# Save untrained model
model = convolutional_model()

# Instantiate the callback class
callbacks = myCallback()print(f"Your model was trained for {len(history.epoch)} epochs")

# Train model
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

In [11]:
print(f"Your model was trained for {len(history.epoch)} epochs")

Credit:
* Derived from DeepLearning.ai TensorFlow Developer Professional Certificate Programming Assignment: Improve MNIST with convolutions